In [1]:
import os
import re
import pandas as pd
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging
from tqdm import tqdm
import torch

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = 'london'
review_df = pd.read_csv(f'csv/{dataset}_processed_reviews.csv')

distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
distilbert_pipeline = pipeline("sentiment-analysis", model=distilbert_model, tokenizer=distilbert_tokenizer)

aspects = ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']
weights = {'LABEL_0': -1, 'LABEL_1': 0, 'LABEL_2': 1}
batch_size = 16

aspect_keywords = {
    'cleanliness': ['clean', 'dirty', 'smell', 'stink', 'stunk', 'filthy'],
    'room': ['room', 'bed', 'suite', 'large'],
    'service': ['service', 'staff', 'help', 'support'],
    'location': ['location', 'close', 'area', 'far'],
    'value': ['value', 'worth', 'price'],
    'safety': ['safe', 'safety', 'secure', 'danger', 'dangerous'],
    'comfort': ['comfort', 'comfortable', 'uncomfortable'],
    'transportation': ['bus', 'metro', 'station', 'close', 'walk'],
    'noise': ['sound', 'volume', 'noisy', 'noise']
}

def is_aspect_mentioned(review, aspect):
    keywords = aspect_keywords[aspect]
    return any(keyword in review for keyword in keywords)

for aspect in aspects:
    review_df[f'{aspect}_score'] = 0

def process_reviews(pipeline, weights):
    for aspect in aspects:
        review_df[f'{aspect}_score'] = 0

    for aspect in aspects:
        print(f"Starting processing for aspect: {aspect}")
        for i in tqdm(range(0, len(review_df), batch_size), desc=f"Batches for {aspect}"):
            batch_reviews = review_df['processed_review'][i:i + batch_size]
            batch_index = batch_reviews.index
            aspect_mentioned = [is_aspect_mentioned(review, aspect) for review in batch_reviews]
            if any(aspect_mentioned):
                filtered_reviews = [review for review, mentioned in zip(batch_reviews, aspect_mentioned) if mentioned]
                aspect_reviews = [f"{aspect}: {review[:512 - len(aspect) - 2]}" for review in filtered_reviews]
                results = pipeline(aspect_reviews)
                scores = [weights[result['label']] * result['score'] for result in results]
                score_index = [index for index, mentioned in zip(batch_index, aspect_mentioned) if mentioned]
                review_df.loc[score_index, f'{aspect}_score'] = scores
            else:
                review_df.loc[batch_index, f'{aspect}_score'] = 0
    for aspect in aspects:
        review_df[f'{aspect}_score'] = pd.to_numeric(review_df[f'{aspect}_score'], errors='coerce')

    return review_df

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
review_df = process_reviews(distilbert_pipeline, weights)

Starting processing for aspect: cleanliness


Batches for cleanliness:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_4344\3128758015.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.5148136019706726, -0.5313187837600708, -0.5274197459220886, -0.5092805624008179, -0.5340520143508911, -0.5279263854026794, -0.5268454551696777, -0.5215900540351868, -0.5326671600341797, -0.5379255414009094, -0.5370203256607056, -0.5253347754478455, -0.5296710729598999]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for cleanliness: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


Starting processing for aspect: room


Batches for room:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_4344\3128758015.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.5146633982658386, -0.5291541218757629, -0.5239787697792053, -0.5083619356155396, -0.5281306505203247, -0.5276796817779541, -0.5251350998878479, -0.5254685878753662, -0.5322167277336121, -0.5292683839797974, -0.5373662710189819, -0.5295249819755554, -0.5349624156951904, -0.5342989563941956, -0.5218091607093811, -0.5314130187034607]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for room: 100%|██████████| 4/4 [00:04<00:00,  1.17s/it]


Starting processing for aspect: service


...

Batches for comfort:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_4344\3128758015.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.5314435362815857, -0.5299619436264038, -0.5174782276153564]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for comfort: 100%|██████████| 4/4 [00:00<00:00,  6.40it/s]


Starting processing for aspect: transportation


Batches for transportation:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_4344\3128758015.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.5164074897766113, -0.5304802060127258, -0.5365999937057495, -0.5302020907402039, -0.5300411581993103, -0.5255721211433411, -0.5359911322593689, -0.53461754322052, -0.5389713048934937, -0.535763680934906, -0.5282996296882629]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for transportation: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


Starting processing for aspect: noise


Batches for noise:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_4344\3128758015.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.5171375870704651, -0.5356610417366028]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for noise: 100%|██████████| 4/4 [00:01<00:00,  3.95it/s]


In [5]:
def map_score_to_label(score, negative_threshold=-0.2, positive_threshold=0.2):
    if score < negative_threshold:
        return -1
    elif score > positive_threshold:
        return 1
    else:
        return 0


In [6]:
import pandas as pd
annotated_df = pd.read_excel('xlsx/london_derlenmis_reviews.xlsx')
annotated_df = annotated_df.drop('hotel_review', axis=1)

reviewdf = review_df
reviewdf = reviewdf.drop('processed_review', axis=1)

In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ensure both DataFrames are sorted similarly
reviewdf = reviewdf.sort_values(by=['hotel_name', 'hotel_city']).reset_index(drop=True)
annotated_df = annotated_df.sort_values(by=['hotel_name', 'hotel_city']).reset_index(drop=True)

# Map DeBERTa scores to labels
for aspect in ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']:
    reviewdf[f'{aspect}_label'] = reviewdf[f'{aspect}_score'].apply(map_score_to_label)

accuracy_mean = 0
precision_mean = 0
recall_mean = 0
f1_mean = 0
inc = 0
# Compare predictions to ground truth
for aspect in ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']:
    accuracy = accuracy_score(annotated_df[f'{aspect}_score'], reviewdf[f'{aspect}_label'])
    precision = precision_score(annotated_df[f'{aspect}_score'], reviewdf[f'{aspect}_label'], average='weighted')
    recall = recall_score(annotated_df[f'{aspect}_score'], reviewdf[f'{aspect}_label'], average='weighted')
    f1 = f1_score(annotated_df[f'{aspect}_score'], reviewdf[f'{aspect}_label'], average='weighted')
    '''
    print(f'Aspect: {aspect}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}\n')
    '''
    accuracy_mean += accuracy
    precision_mean += precision
    recall_mean += recall
    f1_mean += f1
print(f'accuracy_mean: {accuracy_mean/9}')
print(f'precision_mean: {precision_mean/9}')
print(f'recall_mean: {recall_mean/9}')
print(f'f1_mean: {f1_mean/9}')

accuracy_mean: 0.46236559139784944
precision_mean: 0.4401150437274218
recall_mean: 0.46236559139784944
f1_mean: 0.4273133047229073


c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo